# SCohenLab 2D BATCH Image Processing notebook (Simplified MCZ)
--------------

# PIPELINE OVERVIEW
## ❶ GOAL SETTING ✍


### GOAL:  Infer sub-cellular components in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components (Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and CELLMASK) during differentiation of iPSCs, in order to understand the Interactome / Spatiotemporal coordination.

### summary of _OBJECTIVES_ ✅
- robust inference of subcellular objects:
  + 1️⃣-***nuclei***
  + 2️⃣-***cellmask***
  + 3️⃣-***cytoplasm*** (+ ***nucleus***)
  + 4️⃣-***lysosome***
  + 5️⃣-***mitochondria***
  + 6️⃣-***golgi***
  + 7️⃣-***peroxisome***
  + 8️⃣-***ER***
  + 9️⃣-***lipid droplet***





## ❷ DATA CREATION
> METHODS:📚📚
> 
> iPSC lines prepared and visualized on Zeiss Microscopes. 32 channel multispectral images collected.  Linear Unmixing in  ZEN Blue software with target emission spectra yields 8 channel image outputs.  Channels correspond to: Nuclei (NU), Lysosomes (LS),Mitochondria (MT), Golgi (GL), Peroxisomes (PO), Endoplasmic Reticulum (ER), Lipid Droplet (LD), and a “residual” signal.

> Meta-DATA 🏺 (artifacts)
>   - Microcope settings
>  - OME scheme
> - Experimenter observations
> - Sample, bio-replicate, image numbers, condition values, etc
>  - Dates
>  - File structure, naming conventions
>  - etc.





## ❸. IMAGE PROCESSING ⚙️🩻🔬
### INFERENCE OF SUB-CELLULAR OBJECTS
The imported images have already been pre-processed to transform the 32 channel spectral measuremnts into "linearly unmixed" images which estimate independently labeled sub-cellular components.  Thes 7 channels (plus a residual "non-linear" signal) will be used to infer the shapes and extents of these sub-cellular components.   
We will perform computational image analysis on the pictures (in 2D an 3D) to _segment_ the components of interest for measurement.  In other prcoedures we can used these labels as "ground truth" labels to train machine learning models to automatically perform the inference of these objects.
Pseudo-independent processing of the imported multi-channel image to acheive each of the 9 objecives stated above.  i.e. infering: NUCLEI, CELLMASK, CYTOPLASM, LYSOSOME, MITOCHONDRIA, GOLGI COMPLEX, PEROZISOMES, ENDOPLASMIC RETICULUM, and LIPID BODIES

### General flow for infering objects via segmentation
- Pre-processing 🌒
- Core-processing (thresholding) 🌕
- Post-processing  🌘

### QC 🚧 WIP 🚧 




## ❹. QUANTIFICATION 📏📐🧮

SUBCELLULAR COMPONENT METRICS
-  extent 
-  size
-  shape
-  position



### NOTE: PIPELINE TOOL AND DESIGN CHOICES?
We want to leverage the Allen Cell & Structure Setmenter.  It has been wrapped as a [napari-plugin](https://www.napari-hub.org/plugins/napari-allencell-segmenter) but fore the workflow we are proving out here we will want to call the `aicssegmentation` [package](https://github.com/AllenCell/aics-segmentation) directly.

#### ​The Allen Cell & Structure Segmenter 
​The Allen Cell & Structure Segmenter is a Python-based open source toolkit developed at the Allen Institute for Cell Science for 3D segmentation of intracellular structures in fluorescence microscope images. This toolkit brings together classic image segmentation and iterative deep learning workflows first to generate initial high-quality 3D intracellular structure segmentations and then to easily curate these results to generate the ground truths for building robust and accurate deep learning models. The toolkit takes advantage of the high replicate 3D live cell image data collected at the Allen Institute for Cell Science of over 30 endogenous fluorescently tagged human induced pluripotent stem cell (hiPSC) lines. Each cell line represents a different intracellular structure with one or more distinct localization patterns within undifferentiated hiPS cells and hiPSC-derived cardiomyocytes.

More details about Segmenter can be found at https://allencell.org/segmenter
In order to leverage the A




## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops, label
from skimage.morphology import binary_erosion


import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        export_tiff,
                                        list_image_files)

from infer_subc.core.img import *
from infer_subc.utils.stats import *
from infer_subc.utils.stats_helpers import *


from infer_subc.organelles import * 

import time
%load_ext autoreload
%autoreload 2



In [2]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH )              


## SETUP

In [3]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [4]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

source_file = meta_dict['file_name']

/Users/ergonyc/miniforge3/envs/napari10/lib/python3.10/site-packages/ome_types/_convenience.py:112: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


In [5]:
scale[0]/scale[1], scale

(7.267318290023735,
 (0.5804527163320905, 0.07987165184837318, 0.07987165184837318))

Now get the single "optimal" slice of all our organelle channels....

## get the inferred cellmask, nuclei and cytoplasm objects

(takes < 1 sec)

Builde the segmentations in order




In [6]:

###################
# CELLMASK, NUCLEI, CYTOPLASM, NUCLEUS
###################
nuclei_obj =  get_nuclei(img_data,meta_dict, out_data_path)
cellmask_obj = get_cellmask(img_data, nuclei_obj, meta_dict, out_data_path)
cyto_mask = get_cytoplasm(nuclei_obj , cellmask_obj , meta_dict, out_data_path)



loaded  inferred 3D `nuclei`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `cellmask`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `cytoplasm`  from /Users/ergonyc/Projects/Imaging/data/out 


-------------------------
## regionprops

`skimage.measure.regionprops` provides the basic tools nescessary to quantify our segmentations.

First lets see what works in 3D.  

> Note: the names of the regionprops correspond to the 2D analysis even for those which are well defined in 3D.  i.e. "area" is actually "volume" in 3D, etc.


-----------------
## basic stats

### per-organelle


- regionprops 


### summary stats

- group + aggregate:  surface_area, volume
  - median
  - mean
  - std 
  - count

- normalizers
  - CELLMASK?
  - CYTOPLASM?

### nuclei caveats
The other organelles are sensibly normalized by cytoplasm.  does normalizing the nuclei by cytoplasm make sense?  or use cellmask?

Lets see which possible measures are sensible for 3D or volumetric with regionprops

In [7]:
if False:
    labels = label(nuclei_obj )
    rp = regionprops(labels, intensity_image=img_data[NUC_CH])

    supported = [] 
    unsupported = []

    for prop in rp[0]:
        try:
            rp[0][prop]
            supported.append(prop)
        except NotImplementedError:
            unsupported.append(prop)

    print("Supported properties:")
    print("  " + "\n  ".join(supported))
    print()
    print("Unsupported properties:")
    print("  " + "\n  ".join(unsupported))

Supported properties:
-  area
-  bbox
-  bbox_area
-  centroid
-  convex_area
-  convex_image
-  coords
-  equivalent_diameter
-  euler_number
-  extent
-  feret_diameter_max
-  filled_area
-  filled_image
-  image
-  inertia_tensor
-  inertia_tensor_eigvals
-  intensity_image
-  label
-  local_centroid
-  major_axis_length
-  max_intensity
-  mean_intensity
-  min_intensity
-  minor_axis_length
-  moments
-  moments_central
-  moments_normalized
-  slice
-  solidity
-  weighted_centroid
-  weighted_local_centroid
-  weighted_moments
-  weighted_moments_central
-  weighted_moments_normalized
-
Unsupported properties:
-  eccentricity
-  moments_hu
-  orientation
-  perimeter
-  perimeter_crofton
-  weighted_moments_hu

In [8]:
# from scipy.ndimage import find_objects
    
# labels = label(nuclei_obj ).astype("int")
# objects = find_objects(labels)

# # objects are the slices into the original array for each organelle

In [9]:
# get overall summary stats for cellmask
cm_intensity =  raw_cellmask_fromaggr(img_data, scale_min_max=False)


In [10]:
cellmask_obj.dtype

dtype('uint8')

Now we want to get a list of our organelle names, segmentations, intensities (florescence)

In [34]:
# names of organelles we have
organelle_names = ["nuclei","lyso", "mitochondria","golgi","peroxisome","er","lipid"]

get_methods  = [get_nuclei,
                get_lyso,
            get_mito,
            get_golgi,
            get_perox,
            get_ER,
            get_LD]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [NUC_CH, LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

intensities = [img_data[ch] for ch in organelle_channels]


loaded  inferred 3D `nuclei`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `lyso`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded lyso in (0.00) sec
loaded  inferred 3D `mitochondria`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `golgi`  from /Users/ergonyc/Projects/Imaging/data/out 
starting segmentation...
loaded  inferred 3D `peroxisome`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded peroxisome in (0.00) sec
loaded  inferred 3D `er`  from /Users/ergonyc/Projects/Imaging/data/out 
loaded  inferred 3D `lipid`  from /Users/ergonyc/Projects/Imaging/data/out 



-----------------
## CONTACTS (cross-stats)

### organelle cross stats


- regionprops 



- intersect for A vs all other organelles Bi
  - regionprops on A ∩ Bi

   
- contacts?
  - dilate then intersect?
  - loop through each sub-object for each 



In [35]:


n_files = dump_organelle_stats(organelle_names, organelles,intensities, cyto_mask, out_data_path, source_file)

getting stats for A = nuclei
  b = lyso
  b = mitochondria
  b = golgi
  b = peroxisome
  b = er
  b = lipid
getting stats for A = lyso
  b = nuclei
  b = mitochondria
  b = golgi
  b = peroxisome
  b = er
  b = lipid
getting stats for A = mitochondria
  b = nuclei
  b = lyso
  b = golgi
  b = peroxisome
  b = er
  b = lipid
getting stats for A = golgi
  b = nuclei
  b = lyso
  b = mitochondria
  b = peroxisome
  b = er
  b = lipid
getting stats for A = peroxisome
  b = nuclei
  b = lyso
  b = mitochondria
  b = golgi
  b = er
  b = lipid
getting stats for A = er
  b = nuclei
  b = lyso
  b = mitochondria
  b = golgi
  b = peroxisome
  b = lipid
getting stats for A = lipid
  b = nuclei
  b = lyso
  b = mitochondria
  b = golgi
  b = peroxisome
  b = er
dumped 42 csvs


In [36]:
n_files = dump_shell_cross_stats(organelle_names, organelles, cyto_mask, out_data_path, source_file) 


getting stats for A = nuclei
  X lyso
  X mitochondria
  X golgi
  X peroxisome
  X er
  X lipid
getting stats for A = lyso
  X nuclei
  X mitochondria
  X golgi
  X peroxisome
  X er
  X lipid
getting stats for A = mitochondria
  X nuclei
  X lyso
  X golgi
  X peroxisome
  X er
  X lipid
getting stats for A = golgi
  X nuclei
  X lyso
  X mitochondria
  X peroxisome
  X er
  X lipid
getting stats for A = peroxisome
  X nuclei
  X lyso
  X mitochondria
  X golgi
  X er
  X lipid
getting stats for A = er
  X nuclei
  X lyso
  X mitochondria
  X golgi
  X peroxisome
  X lipid
getting stats for A = lipid
  X nuclei
  X lyso
  X mitochondria
  X golgi
  X peroxisome
  X er



-----------------
##  SUMMARY STATS  
> WARNING: (🚨🚨🚨🚨 WIP)
### normalizations.

- overlaps, normalized by CYTOPLASM, A, and B
- per cell averages, medians, std, and totals

These is all pandas munging and very straightforward tabular manipulation.


In [37]:
target = organelle_names[1]

csv_path = out_data_path / f"{source_file.stem}-{target}-stats.csv"

mito_table = pd.read_csv(csv_path)
mito_table.head()

,Unnamed: 0,label,max_intensity,mean_intensity,min_intensity,volume,equivalent_diameter,centroid-0,centroid-1,centroid-2,...,mitochondria_overlap,mitochondria_labels,golgi_overlap,golgi_labels,peroxisome_overlap,peroxisome_labels,er_overlap,er_labels,lipid_overlap,lipid_labels
0,0,1,6618,2426.684211,0,76,5.255390,1.328947,149.578947,665.907895,...,0,[],0,[],0,[],0,[],0,[]
1,1,2,65535,16752.316071,0,30857,38.915119,6.450530,206.110121,517.963801,...,3602,"[6, 47, 48, 49, 70, 80, 88, 114, 130, 133, 151...",2019,[1],77,"[3, 5, 6, 15, 17, 20, 21]",3740,"[3, 19, 187, 201]",0,[]
2,2,5,21639,10844.098901,2242,1547,14.349295,2.798319,370.100194,595.351648,...,0,[],0,[],0,[],139,"[3, 77]",0,[]
3,3,6,5773,2776.759036,480,83,5.412025,1.445783,131.614458,569.843373,...,0,[],0,[],0,[],0,[],0,[]
4,4,7,11263,3762.868852,482,61,4.884016,1.000000,133.180328,604.426230,...,0,[],0,[],0,[],0,[],0,[]


In [38]:
mito_table.volume.mean()

480.036231884058


-----------------
## DISTRIBUTION  
> WARNING: (🚨🚨🚨🚨 WIP)
### XY- summary
Segment image in 3D;
sum projection of binary image; 
create 5 concentric rings going from the edge of the nuclie to the edge of the cellmask (ideally these will be morphed to cellmask/nuclei shape as done in CellProfiler); 
measure intensity per ring (include nuclei as the center area to measure from)/ring area; 
the normalized measurement will act as a frequency distribution of that organelle starting from the nuclei bin going out to the cell membrane - 
Measurements needed: mean, median, and standard deviation of the frequency will be calculated

- pre-processing
  1. Make 2D sum projection of binary segmentation
  2. Create 5 concentric rings going out from the edge of the nuclei to the edge of the cellmask - these rings should be morphed to the shape of the nuclei and cellmask. 
  3. Use nucleus + concentric rings to mask the 2D sum project into radial distribution regions: nuclei = bin 1, ... largest/outter most ring = bin 6. See similar concept in CellProfiler: https://cellprofiler-manual.s3.amazonaws.com/CellProfiler-4.2.5/modules/measurement.html?highlight=distribution#module-cellprofiler.modules.measureobjectintensitydistribution"	
   
- per-object measurements
  - For each bin measure:
    1. pixel ""intensity""
    2. bin area"

- per-object calculations
  - per-object For each bin: 
  - sum of pixel intensity per bin / bin area"	

- per cell summary
  1. Create a frequency table with bin number of the x axis and normalized pixel intensity on the y-axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"



In [39]:
# csv_path = out_data_path / f"{o}_{meta_dict["file_name"].split('/')[-1].split('.')[0]}_stats.csv"
Path(meta_dict['file_name']).name

'ZSTACK_PBTOhNGN2hiPSCs_BR1_N19_Unmixed.czi'

In [40]:
organelles[0].shape, organelle_names[0]

((15, 768, 768), 'nuclei')

In [41]:
test_org = 0

# args 
cellmask_obj
nuclei_obj
organelle_mask = cyto_mask
organelle_name = organelle_names[test_org]
organelle_obj = organelles[test_org]
organelle_img = intensities[test_org]



In [42]:
rad_stats, bin_idx = get_proj_XYstats(        
        cellmask_obj,
        organelle_mask,
        organelle_obj,
        organelle_img,
        organelle_name,
        nuclei_obj
        )


In [43]:
rad_stats

,organelle,mask,bin,n_bins,n_pix,cm_vox_cnt,org_vox_cnt,org_intensity,cm_radial_cv,org_radial_cv,img_radial_cv
0,nuclei,cellmask,Ctr,5,43090.0,560133.0,65727.0,1.231449e+09,0.039089,0.129276,0.106966
1,nuclei,cellmask,1,5,13370.0,133005.0,0.0,2.167817e+08,0.238991,0.000000,0.327669
2,nuclei,cellmask,2,5,19158.0,157767.0,0.0,2.244383e+08,0.383749,0.000000,0.388063
3,nuclei,cellmask,3,5,29521.0,181294.0,0.0,2.700947e+08,0.463339,0.000000,0.409650
4,nuclei,cellmask,4,5,75192.0,204117.0,0.0,3.895217e+08,0.354069,0.000000,0.319295


In [44]:
viewer = napari.view_image(bin_idx)


### Z- summary
Segment image in 3D;
measure area fraction of each organelle per Z slice;
these measurements will act as a frequency distribution of that organelle starting from the bottom of the cellmask (not including neurites) to the top of the cellmask;
measurements: mean, median, and standard deviation of the frequency distribution	

- pre-processing
  1. subtract nuclei from the cellmask --> cellmask cytoplasm
  2. mask organelle channels with cellmask cytoplasm mask

- per-object measurements
  - For each Z slice in the masked binary image measure:
    1. organelle area
    2. cellmask cytoplasm area

- per-object calculations
  - For each Z slice in the masked binary image: organelle area / cellmask cytoplasm area

- per cell summary
  1. create a frequency table with the z slice number on the x axis and the area fraction on the y axis
  2. Measure the frequency distribution's mean, median, and standard deviation for each cell"

In [45]:
viewer.add_image(cellmask_obj>0)

<Image layer 'Image' at 0x2bd9b6a10>

In [46]:


# flattened
cellmask_proj = create_masked_proj_on_Z(cellmask_obj)
org_proj = create_masked_proj_on_Z(organelle_obj,organelle_mask.astype(bool))
img_proj = create_masked_proj_on_Z(organelle_img,organelle_mask.astype(bool), to_bool=False)

nucleus_proj = create_masked_proj_on_Z(nuclei_obj,cellmask_obj.astype(bool)) if nuclei_obj is not None else None


In [47]:
z_stats = get_proj_Zstats(        
        cellmask_obj,
        organelle_mask,
        organelle_obj,
        organelle_img,
        organelle_name,
        nuclei_obj
        )

In [55]:
cellmask_proj = create_masked_proj_on_Z(cellmask_obj)
org_proj = create_masked_proj_on_Z(organelle_obj,organelle_mask.astype(bool))
img_proj = create_masked_proj_on_Z(organelle_img,organelle_mask.astype(bool), to_bool=False)

nucleus_proj = create_masked_proj_on_Z(nuclei_obj,cellmask_obj.astype(bool)) if nuclei_obj is not None else None


In [48]:
z_stats

,organelle,mask,bin,n_bins,cm_vox_cnt,org_vox_cnt,org_intensity,nuc_vox_cnt
0,nuclei,cellmask,0,15,0,0,0,0
1,nuclei,cellmask,1,15,149379,17669,446015391,17669
2,nuclei,cellmask,2,15,121726,6536,247810538,24205
3,nuclei,cellmask,3,15,109521,4851,181560714,29056
4,nuclei,cellmask,4,15,99450,2164,126591321,31212
5,nuclei,cellmask,5,15,94083,1470,105567356,32536
6,nuclei,cellmask,6,15,90942,1063,94683190,32729
7,nuclei,cellmask,7,15,93625,1717,107276890,32333
8,nuclei,cellmask,8,15,90628,3771,125758228,30862
9,nuclei,cellmask,9,15,85107,7398,161987403,27104


In [49]:
n_files = dump_projection_stats(organelle_names, organelles,intensities, cellmask_obj, nuclei_obj, cyto_mask, out_data_path, source_file)

dumped 7x2 csvs


In [50]:
n_files

7

In [51]:
organelle_names

['nuclei', 'lyso', 'mitochondria', 'golgi', 'peroxisome', 'er', 'lipid']